In [8]:
import requests
from bs4 import BeautifulSoup
import seaborn as sns
from datetime import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
user_data_dir = "/Users/emreyuce/Library/Application Support/Google/Chrome/User Data"

# Options to specify the user profile
options = webdriver.ChromeOptions()
options.add_argument(f"user-data-dir={user_data_dir}")
options.add_argument("profile-directory=Default")  # 'Default' is the default profile

# Launch Chrome with your profile
driver = webdriver.Chrome(options=options)



# Login URL
login_url = 'https://sso.accounts.dowjones.com/login-page?client_id=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO&redirect_uri=https%3A%2F%2Fwww.wsj.com%2Fclient%2Fauth&response_type=code&scope=openid%20idp_id%20roles%20tags%20email%20given_name%20family_name%20uuid%20djid%20djUsername%20djStatus%20trackid%20prts%20updated_at%20created_at%20offline_access&ui_locales=en-us-x-wsj-223-2&nonce=cdb92d13-ac4b-42a8-8d0a-948fa4ea0c1c&state=yNWtWfVzrx-k-VtM.4ge4KPFVs5H_hxxSzg9zDJ225bvbVOvT5x_aKxkoFY0&resource=https%253A%252F%252Fwww.wsj.com&protocol=oauth2&client=5hssEAdMy0mJTICnJNvC9TXEw3Va7jfO#/signin-password'
driver.get(login_url)

# WebDriverWait kullanarak elementlerin görünür olmasını bekle
wait = WebDriverWait(driver, 10)

# Giriş sayfası yüklendikten sonra kullanıcı adı ve şifreyi gönder
username = wait.until(EC.presence_of_element_located((By.ID, 'emailOrUsername')))
password = wait.until(EC.presence_of_element_located((By.ID, 'password')))

# Giriş bilgilerini gir
username.send_keys('eyuce@mail.ccsf.edu')
password.send_keys('Bin.98Seven!')

# "Sign In" butonunu bul ve tıkla
signin_button = wait.until(EC.element_to_be_clickable((By.ID, 'signin-pass-submit-btn')))
signin_button.click()

# Sayfanın yüklenmesini bekle
time.sleep(1)

# Click the continue button
continue_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "inline-flex")))
continue_button.click()
time.sleep(5)

# Create an empty list to store article data
article_data = []

# Scraping articles in batches of 50 pages
for i in range(1, 501, 50):  # Loop through pages in chunks of 50
    for page in range(i, min(i + 50, 501)):  # Handle 50 pages in each batch, up to 500 pages
        time.sleep(1)
        # Open the news URL for the current page
        news_url = f'https://www.wsj.com/search?query=us%20finance&isToggleOn=true&operator=OR&sort=relevance&duration=4y&startDate=2021%2F01%2F31&endDate=2025%2F01%2F31&meta=&source=wsjie%2Cwsjpro%2Clivecoverage%2Cwsjsitesrch&page={page}'
        driver.get(news_url)

        # Get the page content
        html = driver.page_source

        # Process the page with BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Extract article headlines and dates
        articles = soup.find_all('article', class_='WSJTheme--story--XB4V2mLz')

        for article in articles:
            headline = article.find('div', class_='WSJTheme--search-combined-headline-summary--1bmOvoTg')

            # Extract date
            date_container = article.find('div', class_='WSJTheme--search-combined-byline-timestamp--kmgehrO6')
            date = date_container.find('p', class_='WSJTheme--timestamp--22sfkNDv') if date_container else None

            # Store headline and date in the list
            headline_text = headline.text.strip() if headline else "Başlık bulunamadı."
            date_text = date.text.strip() if date else "Tarih bulunamadı."

            article_data.append({
                'Headline': headline_text,
                'Date': date_text
            })

    # After scraping 50 pages, save the data to CSV
    df_batch = pd.DataFrame(article_data)
    df_batch.to_csv(f'wsj_articles_batch_{i}_{min(i + 49, 500)}.csv', index=False)

# Combine all batches into one DataFrame
df_combined = pd.concat([pd.read_csv(f'wsj_articles_batch_{i}_{min(i + 49, 500)}.csv') for i in range(1, 501, 50)], ignore_index=True)

# Save the combined DataFrame to a single CSV file
df_combined.to_csv('wsj_articles_combined.csv', index=False)

# Close the browser
driver.quit()

